# 프로젝트 : 얼굴 인식하여 캐릭터 씌우기

## Face Detection vs. Face Recognition

### mediapipe - Face Detection

### mediapipe 설치
> pip install mediapipe

## 눈코 위치 잡기

In [28]:
import cv2
import mediapipe as mp

# 얼굴을 찾고, 찾은 얼굴에 표시를 하기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 drawing_utils 모듈 사용

# 동영상 파일 열기
cap = cv2.VideoCapture('face_video.mp4')

                                         # 0(가까운거) or 1(먼거)         0~1(threshold 값)
with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
        while cap.isOpened():
            success, image = cap.read()

            if not success:
                break

            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = face_detection.process(image)

            # Draw the face detection annotations on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            if results.detections:
                # 6개의 얼굴 특징 detection : 오른 눈, 왼 눈, 코 끝, 입 중심, 오른 귀, 왼 귀 (귀구슬점)
                for detection in results.detections:
                    # mp_drawing.draw_detection(image, detection)
                    # print(detection)
                    
                    # 특정 위치 가져오기
                    keypoints = detection.location_data.relative_keypoints
                    right_eye = keypoints[0]
                    left_eye = keypoints[1]
                    nose_tip = keypoints[2]
                    
                    h, w, _ = image.shape # height, width, channel
                    right_eye = (int(right_eye.x * w), int(right_eye.y * h)) # 이미지 내에서 실제 좌표 (x, y)
                    left_eye = (int(left_eye.x * w), int(left_eye.y * h))
                    nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                    
                    # 양 눈에 동그라미 그리기
                    cv2.circle(image, right_eye, 50, (255, 0, 0), 10, cv2.LINE_AA)  # 파란색
                    cv2.circle(image, left_eye, 50, (0, 255, 0), 10, cv2.LINE_AA) # 초록색
                    # 코에 동그라미 그리기
                    cv2.circle(image, nose_tip, 75, (0, 255, 255), 10, cv2.LINE_AA) # 노란색
                    

            cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))
            if cv2.waitKey(1) == ord('q'):
                break
        
cap.release()
cv2.destroyAllWindows()

## 눈코 덮어씌우기 - 그림판 이미지

In [30]:
import cv2
import mediapipe as mp

# 얼굴을 찾고, 찾은 얼굴에 표시를 하기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 drawing_utils 모듈 사용

# 동영상 파일 열기
cap = cv2.VideoCapture('face_video.mp4')
# 이미지 불러오기
image_right_eye = cv2.imread('right_eye.png') # 100x100
image_left_eye = cv2.imread('left_eye.png')  # 100x100
image_nose = cv2.imread('nose.png') # 300x100

                                         # 0(가까운거) or 1(먼거)         0~1(threshold 값)
with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
        while cap.isOpened():
            success, image = cap.read()

            if not success:
                break

            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = face_detection.process(image)

            # Draw the face detection annotations on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            if results.detections:
                # 6개의 얼굴 특징 detection : 오른 눈, 왼 눈, 코 끝, 입 중심, 오른 귀, 왼 귀 (귀구슬점)
                for detection in results.detections:
                    # mp_drawing.draw_detection(image, detection)
                    # print(detection)
                    
                    # 특정 위치 가져오기
                    keypoints = detection.location_data.relative_keypoints
                    right_eye = keypoints[0]
                    left_eye = keypoints[1]
                    nose_tip = keypoints[2]
                    
                    h, w, _ = image.shape # height, width, channel
                    right_eye = (int(right_eye.x * w)-20, int(right_eye.y * h)-100) # 이미지 내에서 실제 좌표 (x, y)
                    left_eye = (int(left_eye.x * w)+20, int(left_eye.y * h)-100)
                    nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                    
                    # 각 특징에 이미지 그리기
                    image[right_eye[1]-50:right_eye[1]+50, right_eye[0]-50:right_eye[0]+50] = image_right_eye
                    # 눈 중심으로 위 아래 50 범위
                    image[left_eye[1]-50:left_eye[1]+50, left_eye[0]-50:left_eye[0]+50] = image_left_eye
                    image[nose_tip[1]-50:nose_tip[1]+50, nose_tip[0]-150:nose_tip[0]+150] = image_nose

            cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))
            if cv2.waitKey(1) == ord('q'):
                break
        
cap.release()
cv2.destroyAllWindows()

## 눈코 덮어씌우기 - 이미지

In [45]:
import cv2
import mediapipe as mp

# 덮어쓰기 함수 정의
def overlay(image, x, y, w, h, overlay_image): # 대상이미지 (3채널), x, y, width, height, 덮어씌울 이미지 (4채널)
    alpha = overlay_image[:,:,3] # BGRA
    mask_image = alpha/255  # 0 ~ 1사이 값으로 (1: 불투명, 0: 투명)
    
    for c in range(0, 3): # 채널 BGR 대상
        image[y-h:y+h, x-w:x+w, c] = (overlay_image[:,:,c] * mask_image) + (image[y-h:y+h, x-w:x+w, c] * (1 - mask_image))

# 얼굴을 찾고, 찾은 얼굴에 표시를 하기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 drawing_utils 모듈 사용

# 동영상 파일 열기
cap = cv2.VideoCapture('face_video.mp4')
# 이미지 불러오기
image_right_eye = cv2.imread('left2.png', cv2.IMREAD_UNCHANGED) 
image_left_eye = cv2.imread('right2.png', cv2.IMREAD_UNCHANGED) 
image_nose = cv2.imread('nose_img2.png', cv2.IMREAD_UNCHANGED) 

                                         # 0(가까운거) or 1(먼거)         0~1(threshold 값)
with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
        while cap.isOpened():
            success, image = cap.read()

            if not success:
                break

            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = face_detection.process(image)

            # Draw the face detection annotations on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            if results.detections:
                # 6개의 얼굴 특징 detection : 오른 눈, 왼 눈, 코 끝, 입 중심, 오른 귀, 왼 귀 (귀구슬점)
                for detection in results.detections:
                    # mp_drawing.draw_detection(image, detection)
                    # print(detection)
                    
                    # 특정 위치 가져오기
                    keypoints = detection.location_data.relative_keypoints
                    right_eye = keypoints[0]
                    left_eye = keypoints[1]
                    nose_tip = keypoints[2]
                    
                    h, w, _ = image.shape # height, width, channel
                    right_eye = (int(right_eye.x * w)-20, int(right_eye.y * h)-100) # 이미지 내에서 실제 좌표 (x, y)
                    left_eye = (int(left_eye.x * w)+20, int(left_eye.y * h)-100)
                    nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                     
                    # image, x, y, w, h, overlay_image
                    overlay(image, *right_eye, 50, 50, image_right_eye)
                    overlay(image, *left_eye, 50, 50, image_left_eye)
                    overlay(image, *nose_tip, 150, 50, image_nose)
                    
                
            cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))
            if cv2.waitKey(1) == ord('q'):
                break
        
cap.release()
cv2.destroyAllWindows()

In [43]:
image_right_eye = cv2.imread('right2.png', cv2.IMREAD_UNCHANGED) 
image_left_eye = cv2.imread('left2.png', cv2.IMREAD_UNCHANGED) 
image_nose = cv2.imread('nose_img2.png', cv2.IMREAD_UNCHANGED) 

image_right_eye.shape, image_left_eye.shape, image_nose.shape

((100, 100, 4), (100, 100, 4), (100, 300, 4))